In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import shutil
import keras
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16

from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

In [14]:
from glob import glob
trn='D:/Penn_Action/*/'
tr= glob(trn)

len(tr)



15

In [15]:
train = []
val = []
test = []
train_y = []
val_y = []
test_y = []

y = 0
for i in tr:
    
    #print(i)
    x = glob(i+'/*/')
    
    #shuffle(x)
    t,tt = train_test_split( x , test_size=0.1, random_state=42)
    t, vv = train_test_split( t , test_size=0.1, random_state=42)
    
    for j in t:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
        
        train.append(j)
        train_y.append(y)
    
    for j in vv:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        val.append(j)
        val_y.append(y)
        
    for j in tt:
        
        mm = len(glob(j+'/*'))
        
        if(mm<10):
            continue
            
        test.append(j)
        test_y.append(y)
        
    y = y+1

from tensorflow.keras.utils import to_categorical
tra_y =  np.array(to_categorical(train_y))
va_y  =  np.array(to_categorical(val_y))
te_y  =  np.array(to_categorical(test_y))

In [16]:
(train, tra_y) = shuffle(train, tra_y)
(val, va_y) = shuffle(val, va_y)
(test, te_y) = shuffle(test, te_y)

In [18]:
import cv2
import numpy as np
import os


# Initialize a numpy array to store the resized images
resized_images = np.zeros((len(test), 10, 64, 64, 3), dtype=np.uint8)

# Loop through each folder in the "test" array
for i, folder in enumerate(test):

    # Loop through the first 10 images in the current folder
    for j, filename in enumerate(os.listdir(folder)[:10]):

        # Load the image and resize it to 64x64 pixels
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.resize(img, (64, 64))

        # Store the resized image in the numpy array
        resized_images[i, j] = img

X_test = resized_images
print("Shape of the resized images array:", np.shape(X_test))

Shape of the resized images array: (239, 10, 64, 64, 3)


In [25]:

def get_te(k , a) :
    x = glob(k+'/*')
    imgdata=[]
    higher = len(x)
    import more_itertools as mit
    
    y = mit.random_combination(range(0, higher), r=10)

    for i in y:
        
        a = Image.open(x[i])
        b = a.resize((64, 64))
        c = np.array(b)
        imgdata.append(c.reshape(64,64,3))
        
    idata = np.array(imgdata)
    X_train = idata
    X_train = X_train.astype('float32') / 255.
    #print(np.shape(X_train))
    return X_train

def get_cat(k) :
    return np.array(k)

In [26]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , labels, batch_size) :
    self.filename = filename
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    y_train = get_cat(batch_y)
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x]), np.array( y_train )

In [27]:
batch_size = 32

my_training_batch_generator = My_Custom_Generator(train, tra_y, batch_size)
my_validation_batch_generator = My_Custom_Generator(val, va_y, batch_size)

In [28]:
class My_Test_Generator(keras.utils.Sequence) :
  
  def __init__(self, filename , batch_size) :
    self.filename = filename
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.filename[idx * self.batch_size : (idx+1) * self.batch_size]
    i=0
    return np.array([get_te(i,self.filename)for i in batch_x])

In [22]:
x = my_training_batch_generator.__getitem__(3)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

num_frames = 10
frame_height = 64
frame_width = 64
num_channels = 3
num_classes = 15  # Change this to the number of classes in your dataset

model = Sequential()

# CNN
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(num_frames, frame_height, frame_width, num_channels)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))

# LSTM
model.add(LSTM(25))

# Fully connected layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 10, 62, 62, 32)   896       
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 10, 31, 31, 32)   0         
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 10, 29, 29, 64)   18496     
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 10, 14, 14, 64)   0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 10, 12, 12, 128) 

In [33]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.9,
    staircase=True)
optimizer = keras.optimizers.Adam(lr_schedule)

In [34]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics='accuracy')
model.fit_generator(generator=my_training_batch_generator, epochs = 30,validation_data = my_validation_batch_generator)

C:\Users\chowd\AppData\Local\Temp\ipykernel_4068\1675710652.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=my_training_batch_generator, epochs = 30,validation_data = my_validation_batch_generator)
C:\Users\chowd\AppData\Local\Temp\ipykernel_4068\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/30
59/59 [==============================] - 104s 2s/step - loss: 2.6161 - accuracy: 0.1242 - val_loss: 2.5594 - val_accuracy: 0.1682
Epoch 2/30
59/59 [==============================] - 90s 2s/step - loss: 2.3596 - accuracy: 0.2148 - val_loss: 2.3306 - val_accuracy: 0.2430
Epoch 3/30
59/59 [==============================] - 89s 2s/step - loss: 2.1458 - accuracy: 0.3038 - val_loss: 2.2042 - val_accuracy: 0.2617
Epoch 4/30
59/59 [==============================] - 88s 1s/step - loss: 1.9733 - accuracy: 0.3737 - val_loss: 1.9606 - val_accuracy: 0.3832
Epoch 5/30
59/59 [==============================] - 88s 1s/step - loss: 1.7683 - accuracy: 0.4350 - val_loss: 1.8846 - val_accuracy: 0.4112
Epoch 6/30
59/59 [==============================] - 87s 1s/step - loss: 1.5436 - accuracy: 0.5096 - val_loss: 1.7927 - val_accuracy: 0.4393
Epoch 7/30
59/59 [==============================] - 85s 1s/step - loss: 1.4046 - accuracy: 0.5741 - val_loss: 1.8585 - val_accuracy: 0.4486
Epoch 8/30
59/59 [=

In [48]:
from numpy import argmax
from sklearn.metrics import accuracy_score
predictions = model.predict(X_test)
p = np.argmax(predictions, axis=1)
yy = np.argmax(te_y, axis=1)
score = accuracy_score(yy, p)
                            
score

0.26359832635983266

In [49]:
#Patch Encoder with Conv2D ,  LSTM , Pos_Emd

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection =keras.Sequential(
            [
                (layers.Conv2D(2, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                TimeDistributed(Flatten()),
                layers.Dense(projection_dim)
            ]) 
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 

In [50]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models

inputs = layers.Input(shape= (10,64,64,3) )

encoded_patches = (PatchEncoder(10, 32 )) (inputs)

for _ in range(6):
    
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    attention_output= layers.MultiHeadAttention (  num_heads=6, key_dim=32, dropout=0.1 )  (x1, x1)

    x2 = layers.Add()([attention_output, encoded_patches])

    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)

    x3 = keras.Sequential(layers.Dense(units=32, activation=tf.nn.gelu) )(x3)

    encoded_patches = layers.Add()([x3, x2])

representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

representation = layers.GlobalAvgPool1D()(representation)

outputs = layers.Dense(units=15, activation="softmax") ( representation)

model1 = keras.Model(inputs=inputs, outputs=outputs)
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10, 64, 64,  0           []                               
                                 3)]                                                              
                                                                                                  
 patch_encoder_2 (PatchEncoder)  (None, 10, 32)      61912       ['input_3[0][0]']                
                                                                                                  
 layer_normalization_13 (LayerN  (None, 10, 32)      64          ['patch_encoder_2[0][0]']        
 ormalization)                                                                                    
                                                                                            

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.9,
    staircase=True)
optimizer = keras.optimizers.Adam(lr_schedule)

model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics='accuracy')
model1.fit_generator(generator=my_training_batch_generator, epochs = 30,validation_data = my_validation_batch_generator)

C:\Users\chowd\AppData\Local\Temp\ipykernel_4068\2365464098.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model1.fit_generator(generator=my_training_batch_generator, epochs = 30,validation_data = my_validation_batch_generator)
C:\Users\chowd\AppData\Local\Temp\ipykernel_4068\2168406815.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (np.ceil(len(self.filename) / float(self.batch_size))).astype(np.int)


Epoch 1/30
59/59 [==============================] - 100s 2s/step - loss: 2.6172 - accuracy: 0.1311 - val_loss: 2.4626 - val_accuracy: 0.1449
Epoch 2/30
59/59 [==============================] - 89s 2s/step - loss: 2.3299 - accuracy: 0.2020 - val_loss: 2.4251 - val_accuracy: 0.2056
Epoch 3/30
59/59 [==============================] - 103s 2s/step - loss: 2.0590 - accuracy: 0.3092 - val_loss: 2.4816 - val_accuracy: 0.2150
Epoch 4/30
59/59 [==============================] - 102s 2s/step - loss: 1.7518 - accuracy: 0.4190 - val_loss: 2.6959 - val_accuracy: 0.2103
Epoch 5/30
59/59 [==============================] - 100s 2s/step - loss: 1.4528 - accuracy: 0.5325 - val_loss: 2.9788 - val_accuracy: 0.2103
Epoch 6/30
59/59 [==============================] - 95s 2s/step - loss: 1.1714 - accuracy: 0.6301 - val_loss: 3.1219 - val_accuracy: 0.2243
Epoch 7/30
59/59 [==============================] - 92s 2s/step - loss: 0.9661 - accuracy: 0.6962 - val_loss: 3.1985 - val_accuracy: 0.2477
Epoch 8/30
59/59

In [ ]:
from numpy import argmax
from sklearn.metrics import accuracy_score
predictions = model.predict(X_test)
p = np.argmax(predictions, axis=1)
yy = np.argmax(te_y, axis=1)
score = accuracy_score(yy, p)
                            
score